In [1]:
import numpy as np
import orquestra.sdk as sdk
import pandas as pd
import s3fs, os

In [2]:
config_name = 'darpa-ta1'
workspace_id = 'ta1-chemistry-1a2981'

key = sdk.secrets.get(
    "darpa-shared-data-access-key-id",
    config_name=config_name,
    workspace_id=workspace_id,
)
secret = sdk.secrets.get(
    "darpa-shared-data-secret-access-key",
    config_name=config_name,
    workspace_id=workspace_id,
)

s3 = s3fs.S3FileSystem(key=key, secret=secret)

In [3]:
cat_meta = pd.read_csv('catalysis_metadata_new_fcidump.csv')
#cat_meta.head(2)

In [4]:
S3_BUCKET = "darpa-benchmarking-backup-2/zap-storage-zapata-darpa-benchmarking-426lh/scf_outputs/"

In [14]:
for _, row in cat_meta.iterrows():
    #print(row['molecule_name'])
    ru_macho_non_tm = (row['cat_tag'] == 'ru_macho' and (row['molecule_name'] == 'co2' or row['molecule_name'] == 'melact'))
    if ru_macho_non_tm or row['cat_tag'] == 'fe_red':
        fcidump_name = f'fcidump.{row["id"]}_{row["molecule_name"]}_0.2_old'
        s3.download(f'{S3_BUCKET}fcidump.{row["id"]}_{row["molecule_name"]}_*', ".")
        os.rename(row['mean_field_obejct_from_fcidump'], fcidump_name)
    else:
        remote_name = S3_BUCKET + row['mean_field_obejct_from_fcidump']
        s3.download(remote_name, row['mean_field_obejct_from_fcidump'])

In [5]:
#cat_meta.query('cat_tag == "ru_macho"')

In [6]:
def rename_old_instances(row):
    ru_macho_non_tm = (row['cat_tag'] == 'ru_macho' and (row['molecule_name'] == 'co2' or row['molecule_name'] == 'melact'))
    if ru_macho_non_tm or row['cat_tag'] == 'fe_red':
        return f'fcidump.{row["id"]}_{row["molecule_name"]}_0.2_old'
    else: 
        return row['mean_field_obejct_from_fcidump']

In [17]:
cat_meta.loc[:, 'mean_field_obejct_from_fcidump'] = cat_meta.apply(rename_old_instances, axis=1)

In [18]:
cat_meta.query('cat_tag == "ru_macho"')[['molecule_name', 'mean_field_obejct_from_fcidump']]

,molecule_name,mean_field_obejct_from_fcidump
0,co2,fcidump.2_co2_0.2_old
1,melact,fcidump.7_melact_0.2_old
2,melact,fcidump.8_melact_0.2_old
12,ru_macho,fcidump.0_ru_macho_noncan_0.2_new
23,ru_macho,fcidump.1_ru_macho_noncan_0.2_new
44,ts_ru_macho_co2,fcidump.3_ts_ru_macho_co2_noncan_0.2_new
55,ts_ru_macho_co2,fcidump.4_ts_ru_macho_co2_noncan_0.2_new
57,ts_ru_macho_melact,fcidump.5_ts_ru_macho_melact_noncan_0.2_new
65,ts_ru_macho_melact,fcidump.6_ts_ru_macho_melact_noncan_0.2_new


In [19]:
cat_meta.to_csv('catalysis_metadata_new_fcidump.csv', index=False)

In [7]:
cat_meta.columns

Index(['geometry', 'basis_set', 'charge', 'multiplicity',
       'avas_atomic_orbitals', 'avas_minao', 'method_type', 'molecule_name',
       'ref', 'cat_tag', 'category', 'id', 'nbasis', 'chk_list', 'id_list',
       'num_files', 'mean_field_obejct_from_fcidump', 'converged',
       'stable_int', 'mult_mf', 's2_mf', 'e_tot', 'avas_ne', 'avas_no',
       'utility_scale', 'software_used'],
      dtype='object')

In [ ]:
# Download and rename the orbital files
for _, row in cat_meta.iterrows():
    chk_name = eval(row['chk_list'])[0]
    ru_macho_non_tm = (row['cat_tag'] == 'ru_macho' and (row['molecule_name'] == 'co2' or row['molecule_name'] == 'melact'))
    if ru_macho_non_tm:
        fcidump_name = f'fcidump.{row["id"]}_{row["molecule_name"]}_0.2_old'
        s3.download(f'{S3_BUCKET}fcidump.{row["id"]}_{row["molecule_name"]}_*', ".")
        os.rename(row['mean_field_obejct_from_fcidump'], fcidump_name)
    else:
        remote_name = S3_BUCKET + row['mean_field_obejct_from_fcidump']
        s3.download(remote_name, row['mean_field_obejct_from_fcidump'])